<a href="https://colab.research.google.com/github/Sohammhatre10/Text-Summarization-Suvidha-Foundation/blob/main/TextSummarizationSuvidha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.0 MB/s eta 0:00:0

In [2]:
from transformers import pipeline, set_seed
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version="3.0.0")
print(f"features in cnn_dailymail : {dataset['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

features in cnn_dailymail : ['article', 'highlights', 'id']


In [4]:
sample = dataset["train"][1]
print(f'{len(sample["article"])}')

4051


In [5]:
sample_text = dataset["train"][1]["article"][:1000]

summarize = {}

In [7]:
def baseline_summary_three_sent(text):
  return "\n".join(sent_tokenize(text)[:3])
summarize['baseline'] = baseline_summary_three_sent(sample_text)
summarize['baseline']

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'

In [19]:
from transformers import pipeline, set_seed

set_seed(42)

pipe = pipeline('text-generation', model ='gpt2-medium')

gpt2_query = sample_text + "\nTl;DR:\n"

pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces = True)

pipe_out

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and

In [21]:
pipe_out[0]['generated_text'][len(gpt2_query):]

'The Eighth Amendment of Chapter Four of the U.S embody a core principle of democracy: "Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof." One particularly common violation of this principle occurs when government employees engage in, or support, activities that violate their oath(s) --or, in these cases, the First Amendment rights of other individuals to freedom of assembly and speech. In 2005, the U.S. Supreme Court ruled that that a U.S. government employee is entitled to free speech rights at his or her job. One of the first things I did after coming home from school was to make sure that I wrote my new professor. I made sure his name wasn\'t Michael -- a pseudonym for another student. And when I saw my face on the poster that I designed for him, I knew I had written the best-written essayAlpha Alpha Gamma came from the Greek word for "good," and so I created an image of a beautiful forest on the moon. The letter aa came f

In [22]:
summarize['gpt2'] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(gpt2_query):]))

## T5

In [23]:
pipe = pipeline('summarization',model="t5-small")

pipe_out = pipe(sample_text)

pipe_out

[{'summary_text': "inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court . most often, they face drug charges or charges of assaulting an officer . mentally ill people become more paranoid, delusional, and less likely to follow dir ."}]

In [24]:
summarize["t5"] = 'n'.join(sent_tokenize(pipe_out[0]['summary_text']))

# BART

In [26]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")

pipe_out = pipe(sample_text)

pipe_out

[{'summary_text': 'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated. Most often, they face drug charges or charges of assaulting an officer. Judge Steven Leifman says the arrests often result from confrontations with police.'}]

In [28]:
summarize['bart'] = 'n'.join(sent_tokenize(pipe_out[0]['summary_text']))

# PEGASUS

In [29]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")

pipe_out = pipe(sample_text)

pipe_out

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'summary_text': 'Mentally ill inmates are housed on the "forgotten floor" of a Miami jail .<n>Judge Steven Leifman says the charges are usually "avoidable felonies"<n>He says the arrests often result from confrontations with police .<n>Mentally ill people often won\'t do what they\'re told when police arrive on the scene .'}]

In [30]:
summarize['pegasus'] = 'n'.join(sent_tokenize(pipe_out[0]['summary_text']))

# TOTAL

In [33]:
print('Truth')
print(dataset['train'][1]['highlights'])
print("\n")

for model_name in summarize:
  print(model_name.upper())
  print(summarize[model_name])
  print("\n")

Truth
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."


GPT2
The Eighth Amendment of Chapter Four of the U.S embody a core principle of democracy: "Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof."
One particularly common v

## SarceBLEU

In [35]:
from datasets import load_metric

bleu_metric = load_metric('sacrebleu')

<ipython-input-35-07e747a21c2e>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('sacrebleu')


In [38]:
bleu_metric.add(prediction = [summarize["pegasus"]], reference = [dataset['train'][1]['highlights']] )
results = bleu_metric.compute(smooth_method = 'floor', smooth_value=0)

results['precision'] = [np.round(p,2) for p in results['precisions']]

pd.DataFrame.from_dict(results, orient='index', columns=['Value'])

,Value
score,19.784432
counts,"[29, 16, 11, 7]"
totals,"[71, 70, 69, 68]"
precisions,"[40.84507042253521, 22.857142857142858, 15.942..."
bp,1.0
sys_len,71
ref_len,57
precision,"[40.85, 22.86, 15.94, 10.29]"


## ROUGE

In [40]:
rouge_metric = load_metric('rouge')


In [45]:
rouge_names = ["rouge1", 'rouge2', "rougeL", "rougeLsum"]

reference = dataset['train'][1]['highlights']

records = []

for model_name in summarize:
  rouge_metric.add(prediction = summarize[model_name], reference = reference )
  score = rouge_metric.compute()
  rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
  print('rouge_dict', rouge_dict)
  records.append(rouge_dict)

pd.DataFrame.from_records(records, index=summarize.keys())

rouge_dict {'rouge1': 0.365079365079365, 'rouge2': 0.14516129032258066, 'rougeL': 0.20634920634920634, 'rougeLsum': 0.2857142857142857}
rouge_dict {'rouge1': 0.09061488673139159, 'rouge2': 0.013029315960912053, 'rougeL': 0.07119741100323625, 'rougeLsum': 0.09061488673139159}
rouge_dict {'rouge1': 0.1758241758241758, 'rouge2': 0.0, 'rougeL': 0.13186813186813187, 'rougeLsum': 0.15384615384615383}
rouge_dict {'rouge1': 0.3440860215053763, 'rouge2': 0.10989010989010989, 'rougeL': 0.2150537634408602, 'rougeLsum': 0.25806451612903225}
rouge_dict {'rouge1': 0.49019607843137253, 'rouge2': 0.24000000000000002, 'rougeL': 0.3529411764705882, 'rougeLsum': 0.43137254901960786}


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.090615,0.013029,0.071197,0.090615
t5,0.175824,0.000000,0.131868,0.153846
bart,0.344086,0.109890,0.215054,0.258065
pegasus,0.490196,0.240000,0.352941,0.431373
